### This notebook is for model training and save the model to s3 bucket

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import keras

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

2024-04-26 20:02:00.760243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 20:02:00.760301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 20:02:05.324007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 20:02:12.955863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 20:02:30.717293: W tensorflow/comp

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
from tensorflow import keras
import time
s_time = time.time()

#model structure
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

e_time = time.time()
run_time = e_time - s_time

print('\nTest accuracy:', test_acc, '\nRun time:', run_time)


2024-04-26 20:02:49.422929: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10
1875/1875 [==============================] - 20s 9ms/step - loss: 0.6314 - accuracy: 0.8147
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3531 - accuracy: 0.8702
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.3149 - accuracy: 0.8838
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2889 - accuracy: 0.8928
Epoch 5/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2732 - accuracy: 0.8966
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2556 - accuracy: 0.9039
Epoch 7/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2397 - accuracy: 0.9085
Epoch 8/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2259 - accuracy: 0.9150
Epoch 9/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.2136 - accuracy: 0.9177
Epoch 10/10
1875/1875 [==============================] - 17s 9ms/step - l

In [ ]:
import tempfile
import boto3
import joblib

BUCKET_NAME = 'jas01'

s3_client = boto3.client('s3')

# name to save model as in s3
model_name = "model.h5"

# save to s3
with tempfile.TemporaryFile() as fp:
    joblib.dump(model, fp)
    fp.seek(0)
    s3_client.put_object(
        Body=fp.read(), 
        Bucket=BUCKET_NAME, 
        Key=model_name
    )